In [0]:
# %% [code]
%%time
import numpy as np
import pandas as pd
from pathlib import Path
import memory_profiler as mem_profile
import time
import sys

pd.options.mode.chained_assignment = None  # default='warn'

# import data
folder = Path('../input/')
edge_file = folder / 'Copy of edge_data.parquet'
wind_file = folder / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

# %%timeit
def wind_scenario(wind_data):
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(edges=edgelist):
    rng = np.random.uniform(0, 1, size=edges.values.shape[0])
    mask = rng < edges.IgnProb_bl.values
    NewActiveEdges = edges[mask]
    return NewActiveEdges


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges

# %%time
listScenarioDataframes = []
n=1001
for scenario in range(n):
    # initial setup
    condition = True
    # listScenarioDataframes = []
    listActivatedSources = []
    time = 0 
    # wind conditions
    w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
    # ignition / initial state and edges selection
    ActiveEdges = ignition()
    if ActiveEdges.empty:
        continue
    while condition:
        ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
        if ActiveEdges.empty:
            break
        listScenarioDataframes.append(ActiveEdges)
        listActivatedSources.extend(ActiveEdges.source.values)
        ActiveEdges = propagation(ActiveEdges)
        # advance time
        time += 1
    print("scenario : {} time : {} memory : {} MB".format(scenario, time, mem_profile.memory_usage()))
    #     print("size of dataframe list = {}".format(sys.getsizeof(listScenarioDataframes)))
    Activations = pd.concat(listScenarioDataframes)
    Activations.to_parquet('scenario{}_time{}_Activations.parquet'.format(scenario, time), engine='pyarrow')